# Unit 3: Services in ROS

What will you learn with this unit?

1. How to **give a service**
2. Create your **own service server message**

# Part 2: How to give a Service

> * **Service Server:  Contains functionality**

## **Example 3.7**



**Python Program {3.7}: simple_service_server.py**

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Empty, EmptyResponse # you import the service message  
                                     # python classes generated from Empty.srv.


def my_callback(request):
    print "My_callback has been called"
    return EmptyResponse() # the service Response class, in this case EmptyResponse
    #return MyServiceResponse(len(request.words.split())) 

rospy.init_node('service_server') 
my_service = rospy.Service('/my_service', Empty , my_callback)
 # create the Service called my_service with the defined callback

 
rospy.spin() # maintain the service open.

## **1. Call the service**

In [ ]:
rosservice call /my_service [TAB]+[TAB]

In [ ]:

# rospy.Service('/my_service', Empty , my_callback)

rosservice call /my_service "{}"

**Outpput**

In [ ]:
My_callback has been called

 **INFO**: Note that, in the example, there is a commented line in the **my_callback** function. 
 > * This gives you an example of how you would access the request given by the caller of your service.
 > * It's always **request.variables_in_the_request_part_of_srv_message**

## **Exercise 3.2**

1. The objective of Exercise 3.2 is to
> * create a service that, 
> * when called,
>*  will make BB8 robot move in a circle-like trajectory.


2. Create a Service Server that
> * accepts an Empty service message and
> * activates the circle movement. 
> * This service could be called /move_bb8_in_circle.

In [ ]:
bb8_move_in_circle_service_server.py

3. Create a launch file called **start_bb8_move_in_circle_service_server.launch**.
> *  Inside it, you have to start a node that launches the **bb8_move_in_circle_service_server.py** file.

4. Launch **start_bb8_move_in_circle_service_server**.launch and check that, when called through the WebShell, BB-8 moves in a circle.

5. Now, create a new Python file called **bb8_move_in_circle_service_client.py** that calls the service **/move_bb8_in_circle** 
>  Remember how it was done in the previous chapter: **Services Part 1**.

> Then, generate a new launch file called **call_bb8_move_in_circle_service_server**.
>> launch that executes the code in the **bb8_move_in_circle_service_client.py** file.

6. Finally, when you launch this **call_bb8_move_in_circle_service_server.launch** file, 
> BB-8 should move in a circle.


**Python Program: bb8_move_in_circle_service_server.py**



In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Empty, EmptyResponse # you import the service message python classes generated from Empty.srv.
from geometry_msgs.msg import Twist

def my_callback(request):
    rospy.loginfo("The Service move_bb8_in_circle has been called")
    move_circle.linear.x = 0.2
    move_circle.angular.z = 0.2
    my_pub.publish(move_circle)
    rospy.loginfo("Finished service move_bb8_in_circle")
    return EmptyResponse() # the service Response class, in this case EmptyResponse

rospy.init_node('service_move_bb8_in_circle_server') 

# create the Service called move_bb8_in_circle with the defined callback
my_service = rospy.Service('/move_bb8_in_circle', Empty , my_callback) 

my_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
move_circle = Twist()

rospy.loginfo("Service /move_bb8_in_circle Ready")
rospy.spin() # mantain the service open.

In [ ]:
rosservice call /move_bb8_in_circle [TAB]+[TAB]

**Launch Program: start_bb8_move_in_circle_service_server.launch**



In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_circle service -->
    <node pkg="unit_3_services" type="bb8_move_in_circle_service_server.py" name="service_move_bb8_in_circle_server"  output="screen">
    </node>
</launch>

**Python Program: bb8_move_in_circle_service_client.py**

In [ ]:
#! /usr/bin/env python
import rospkg
import rospy
from std_srvs.srv import Empty, EmptyRequest 
 # you import the service message python classes generated from Empty.srv.

rospy.init_node('service_move_bb8_in_circle_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/move_bb8_in_circle') # Wait for the service client /move_bb8_in_circle to be running

move_bb8_in_circle_service_client = rospy.ServiceProxy('/move_bb8_in_circle', Empty) # Create the connection to the service
move_bb8_in_circle_request_object = EmptyRequest() # Create an object of type EmptyRequest

result = move_bb8_in_circle_service_client(move_bb8_in_circle_request_object) # Send through the connection the path to the trajectory file to be executed
print result # Print the result given by the service called

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_circle service -->
    <node pkg="unit_3_services" type="bb8_move_in_circle_service_client.py" name="service_move_bb8_in_circle_client"  output="screen">
    </node>
</launch>

## **2. How to create your own service message**


In [ ]:
roscd;cd ..;cd src
catkin_create_pkg my_custom_srv_msg_pkg rospy

In [ ]:
roscd my_custom_srv_msg_pkg/
mkdir srv

vim srv/MyCustomServiceMessage.srv



```
int32 duration    # The time (in seconds) during which BB-8 will keep moving in circles
---
bool success      # Did it achieve it?
```



### How to Prepare CMakeLists.txt and package.xml for Custom Service Compilation


You have to edit two files in the package similarly to how we explained for Topics:

1. CMakeLists.txt
2. package.xml

1. Modification of CMakeLists.txt
You will have to edit four functions inside CMakeLists.txt:

> 1. find_package()
2. add_service_files()
3. generate_messages()
4. catkin_package()

In [ ]:
find_package(catkin REQUIRED COMPONENTS
  std_msgs
  message_generation
)

In [ ]:
add_service_files(
  FILES
  MyCustomServiceMessage.srv
)

In [ ]:
generate_messages(
  DEPENDENCIES
  std_msgs
)

In [ ]:
catkin_package(
      CATKIN_DEPENDS
      rospy
)

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(my_custom_srv_msg_pkg)



find_package(catkin REQUIRED COMPONENTS
  std_msgs
  message_generation
)


add_service_files(
  FILES
  MyCustomServiceMessage.srv
)

## Here is where the packages needed for the action messages compilation are imported.
generate_messages(
  DEPENDENCIES
  std_msgs
)


catkin_package(
  CATKIN_DEPENDS rospy
)


include_directories(
  ${catkin_INCLUDE_DIRS}
)

### 2. Modification of package.xml:


In [ ]:
 <build_depend>rospy</build_depend>
  <build_export_depend>rospy</build_export_depend>
  <exec_depend>rospy</exec_depend>

  <build_depend>message_generation</build_depend>
  <build_export_depend>message_runtime</build_export_depend>
  <exec_depend>message_runtime</exec_depend>

  <build_depend>std_msgs</build_depend>
  <build_export_depend>std_msgs</build_export_depend>
  <exec_depend>std_msgs</exec_depend>

In [ ]:
roscd;cd ..
catkin_make
source devel/setup.bash

In [ ]:
 rossrv list | grep MyCustomServiceMessage

# my_custom_srv_msg_pkg/MyCustomServiceMessage

That's it! You have created your own Service Message. 
> Now, create a Service Server that uses this type of message.

### **Python Program {3.3}: custom_service_server.py**



In [ ]:
#! /usr/bin/env python

import rospy
from my_custom_srv_msg_pkg.srv import MyCustomServiceMessage, MyCustomServiceMessageResponse # you import the service message python classes 
                                                                                         # generated from MyCustomServiceMessage.srv.


def my_callback(request):
    
    print "Request Data==> duration="+str(request.duration)
    my_response = MyCustomServiceMessageResponse()
    if request.duration > 5.0:
        my_response.success = True
    else:
        my_response.success = False
    return  my_response # the service Response class, in this case MyCustomServiceMessageResponse

rospy.init_node('service_client') 
my_service = rospy.Service('/my_service', MyCustomServiceMessage , my_callback) # create the Service called my_service with the defined callback
rospy.spin() # maintain the service open.